# Exoplanet Exploratory Data Analysis (i.e. exo-eda)

In this data analysis we will analyse a list of currently known exoplanets in the
[NASA Exoplanet Archive](https://exoplanetarchive.ipac.caltech.edu/index.html),
made graciously available to the community on behalf of the KELT project team. For those who do not know, exoplanets are planets that orbit other stars, within our Milky Way galaxy and other galaxies (e.g. the Andromeda Galaxy, the Milky Way's near neighbor).

## Downloading the dataset
The best way to retrieve the data is by using the following piece of code:
It interfaces with the Exoplanet Archive's Table Access Protocol API using the PyVO python package alongside importing all the packages needed for this project. (This usually takes a while.)

In [ ]:
!pip install pyvo pandas seaborn tqdm jovian -Uq
import pandas as pd
from pyvo.dal import TAPService
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from tqdm import tqdm

sns.set_style("dark")

service = TAPService("https://exoplanetarchive.ipac.caltech.edu/TAP")
results = service.search("SELECT * FROM pscomppars")
df: pd.DataFrame = results.to_table().to_pandas(index=True)

## Data Preparation and Cleaning
To be very frank, this is a large dataset. As a result there are many columns in this dataset that we
do not need and take up memory usage.
A quick look through the dataset shows the following:
- Many of the columns contain "reflinks", which HTML references to publications.
In addition, they generally are the same for each column which makes them redundant.
- On a similar note, we do not need to use links that cite the academic papers where the discovery was originally
published
- There is also some columns that contain other HTML, making them unusable (they end with `str`)

In [ ]:
df.info()

In [ ]:
# Deleting the reflinks
df.drop(columns=df.filter(like="reflink").columns, inplace=True)
# Deleting the citation link
df.drop(columns="disc_refname", inplace=True)
# Other HTML columns
# This list also contains disc_instrument, which we need and is at index 0
df.drop(columns=df.filter(like="str").columns[1:], inplace=True)

In [ ]:
df.info(memory_usage=True)

## Exploratory Data Analysis and Visualization

*Gaining an understanding of the dataset*  
For an understanding of what each column refers to, please refer to the [Exoplanet Archive](https://exoplanetarchive.ipac.caltech.edu/docs/API_PS_columns.html)

### Exoplanets

First, it would be helpful to see how many exoplanets that have been detected. 

In [ ]:
print(f'There are {df.shape[0]} exoplanets detected')

### Controversial Exoplanets

Looking at the inclusion of the `pl_controv_flag` it seems as if there are exoplanets whose status is not confirmed. 

In [ ]:
controv_df = pd.DataFrame(
    {"Non-controversial": (df.pl_controv_flag == 0).sum(), "Controversial": (df.pl_controv_flag == 1).sum()}, index=[0])
ax = controv_df.plot(kind="bar")
ax.set_title("Exoplanet Status")
ax.set_xlabel("Current State")
ax.set_ylabel("Count")
for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x() * 1.005, p.get_height() * 1.005))

Since there is a few exoplanets that are (currently) marked as controversial, to be safe, we should create a new dataframe
that only contains confirmed exoplanets.

In [ ]:
non_controv_df = pd.DataFrame(df.iloc[df[df["pl_controv_flag"] == 0].index], columns=df.columns).reset_index(drop=True)

Now we can look further into the dataset and be sure that we are using confirmed data.

### Number of exoplanets discovered each year

Here is a visualization on how many exoplanets we are discovering each year(including this year so far)

In [ ]:
fig, ax = plt.subplots(figsize=(7, 5))
sns.despine(fig)
ax.set_xlabel("Discovery Year")
sns.histplot(non_controv_df["disc_year"].value_counts(), x=non_controv_df["disc_year"].values)
plt.show()

### Planetary Systems


We can also see if there are any planets that orbit multiple stars

In [ ]:
sys_types = non_controv_df["sy_snum"].value_counts()
ax = plt.axes()
ax.bar(sys_types.keys(), sys_types.values)
ax.set_title("Planet Host Star Count")
ax.set_ylabel("# of Planets")
ax.set_xlabel("# of Stars")
plt.xticks([1, 2, 3, 4])
for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x() * 1.005, p.get_height() * 1.005))
plt.show()

### Discovery locale
Have ground or space telescopes found the most exoplanets? (Space telescopes do not have to deal with atmospheric obstructions, e.g. clouds, but are more limited in nature due to the costs/limitations of going to space)

In [1]:
ax=non_controv_df["disc_locale"].value_counts().plot(kind="bar")
ax.set_xlabel("Discovery Locale")
ax.set_ylabel("Count")
ax.set_title("Discovery locale Counts")
plt.show()

NameError: name 'non_controv_df' is not defined

Here we can find out the number of exoplanets that are in a system

In [ ]:
ax=non_controv_df["sy_pnum"].value_counts().plot(kind="bar")
ax.set_xlabel("# of Planets orbiting a star(s)")
ax.set_ylabel("Count")
ax.set_title("# of Exoplanets in a system")
for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x() * 1.005, p.get_height() * 1.005))
plt.show()

This is very interesting to see. It is a good reminder to see that worlds like Tatooine from Star Wars are still possible.

## Asking and answering questions

### Q1: Which techniques have found the most exoplanets?

***To learn more about exoplanet detection techniques, please refer to
[this awesome Wikipedia article](https://en.wikipedia.org/wiki/Methods_of_detecting_exoplanets)***

In [ ]:
disc_method_counts = non_controv_df["discoverymethod"].value_counts()
#ax = plt.axes()
disc_method_counts.plot(kind="bar", title="Discovery Techniques", xlabel="Discovery Method",
                        ylabel="# of Planets found")
plt.show()
print(f'The {str(disc_method_counts.keys()[0]).lower()} method has found the most exoplanets')

### Q2: Which facility found the most exoplanets?

In [ ]:
non_controv_df["disc_facility"].value_counts()

As of writing (July 2021), the Kelper space telescope has found the most exoplanets, with over 2500 exoplanets found
(nearly half of all known exoplanets!). This includes its secondary mission, "K2" after some parts failed, reducing its
capability. It was deactivated in 2018. (It will be interesting to see how many exoplanets TESS finds as it the new
exoplanet hunting space telescope)

### Q3: How far are the exoplanets from their host stars(au)?

An au(astronomical unit) is the average distance from our planet to our sun.
Since we may get some very large numbers for distance, it is better to take the median.

In [ ]:
non_controv_df["pl_orbsmax"].median()

### Q4: What is the median number of planets that orbit a star

In [2]:
non_controv_df["sy_pnum"].median()

NameError: name 'non_controv_df' is not defined

In [ ]:
non_controv_df["sy_pnum"].value_counts()

It seems as if our solar system is quite the exception having 8 planets + 1 (confirmed) dwarf planet, when most have systems have only 1 planet. 

### Q5: How many exoplanets reside in the habitable zone of a star?



A word of caution: this is a non-scientific rough calculation based on the habitable zone of our own sun (which itself is an estimate). Additionally, this only works on planets that orbit only 1 star. Many factors are involved in finding the habitability of our planet. See [this for more details](https://astronomy.stackexchange.com/questions/44974/calculate-the-habitable-zone-of-a-star-based-on-nasa-exoplanet-archive).  
This is based on the [following formulas](https://exoplanetarchive.ipac.caltech.edu/docs/poet_calculations.html) from the Exoplanet Archive.


In [ ]:
def is_habitable(st_lum: float, st_teff: float, st_rad: float, pl_orb: float):
    if not np.isnan(st_lum) and st_lum > 0:
        pass
    elif not (np.isnan(st_rad) or np.isnan(st_teff)):
        st_lum = pow(st_rad, 2) * pow(st_teff, 4)
    else:
        return np.nan
    
    try:
        sqrt_lum = np.sqrt(st_lum)
    except RuntimeWarning:
        return np.nan
    inner_lim = 0.75 * sqrt_lum
    outer_lim = 1.77 * sqrt_lum
    if inner_lim < pl_orb < outer_lim:
        return True
    else:
        return False

In [ ]:
# Filtering to planets that orbit a single star
single_star_df = pd.DataFrame(non_controv_df[non_controv_df["sy_pnum"]==1]).reset_index(drop=True)

In [ ]:
pl_is_in_habitable_range=[]
for i in tqdm(range(single_star_df.shape[0])):
    st_lum = single_star_df["st_lum"][i]
    pl_orb = single_star_df["pl_orbsmax"][i]
    st_teff = single_star_df["st_teff"][i]
    st_rad = single_star_df["st_rad"][i]
    result = is_habitable(st_lum, st_teff,st_rad,pl_orb)
    pl_is_in_habitable_range.append(result)


In [ ]:
single_star_df["pl_is_in_habitable_range"] = pl_is_in_habitable_range
print(f'Only {single_star_df["pl_is_in_habitable_range"].value_counts()[1]} of {single_star_df.shape[0]} exoplanets reside within the habitable zone of their host star')

## Acknowledgements

- Thanks to Jovian for creating awesome courses on Data Science and Machine Learning!
- This data is made available to the community through the Exoplanet Archive on behalf of the KELT project team.